In [ ]:
%load_ext autoreload
%autoreload 2
#!/usr/bin/env python3
import cdsapi
import os
import numpy as np
import subprocess
import era5_tools
import xarray as xr

In [ ]:
def monthly_stats( fp ):
    '''
    Functions to pass to era5_tools.download_multiple_months().
    This will calculate the min, max and mean over each downloaded file.
    '''
    
    # Open the dataset for the input file
    ds = xr.open_dataset(fp, chunks = {'time':1000})['t2m']
    
    # Calculate min and max over all data
    ds_max = ds.max(dim='time')
    ds_min = ds.min(dim='time')
    ds_mean = ds.mean(dim='time')
    ds_dailymax = ds.resample(time = '1D').max
    ds_dailymin = ds.resample(time = '1D').min
    ds_dailymean = ds.resample(time = '1D').mean()
    ds_mean_maxima = ds_dailymax.mean(dim='time')
    ds_mean_minima = ds_dailymin.mean(dim='time')

    # Create output dataset and assign variables
    ds_out = xr.Dataset()
    ds_out['t2m_mean_maxima'] = ds_mean_maxima
    ds_out['t2m_mean_minima'] = ds_mean_minima
    ds_out['t2m_mean'] = ds_mean
    ds_out['t2m_max'] = ds_max
    ds_out['t2m_min'] = ds_min
    return ds_out

In [ ]:
dp_out = '../data/era5'
era5_tools.download_multiple_months('2m_temperature', dp_out, 2000, 2015, 
                                    apply_func=monthly_stats, 
                                    func_name='monthly_stats',
                                    delete_after_func=True)